# Performance Bottlenecks

## Naive Distinct Value Tracking

In [ ]:
import time
import random

def process_ids(ids):
    unique_ids = []
    duplicates = 0
    for id_val in ids:
        if id_val in unique_ids:
            duplicates += 1
        else:
            unique_ids.append(id_val)
    return unique_ids, duplicates

def main():
    # Simulate 100,000 IDs with some duplicates
    random.seed(42)
    ids = [random.randint(1, 100000) for _ in range(100000)]

    start_time = time.time()
    unique, dups = process_ids(ids)
    end_time = time.time()

    print(f"Found {len(unique)} unique IDs and {dups} duplicates in {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Note when I got an AI-generated recommendation ho

In [ ]:
import time
import random

def process_ids(ids):
    seen_ids = set()
    unique_ids = []
    duplicates = 0
    for id_val in ids:
        if id_val in seen_ids:  # Efficient O(1) lookup on set
            duplicates += 1
        else:
            seen_ids.add(id_val)
            unique_ids.append(id_val)  # Maintain order if needed
    return unique_ids, duplicates

def main():
    # Simulate 100,000 IDs with some duplicates
    random.seed(42)
    ids = [random.randint(1, 100000) for _ in range(100000)]

    start_time = time.time()
    unique, dups = process_ids(ids)
    end_time = time.time()

    print(f"Found {len(unique)} unique IDs and {dups} duplicates in {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

The `seen_ids` list was completely redundant to the `seen_ids`.

In [ ]:
import time
import random

def process_ids(ids):
    unique_ids = set()
    duplicates = 0
    for id_val in ids:
        if id_val in unique_ids:  # Efficient O(1) lookup on set
            duplicates += 1
        else:
            unique_ids.add(id_val)
    return unique_ids, duplicates

def main():
    # Simulate 100,000 IDs with some duplicates
    random.seed(42)
    ids = [random.randint(1, 100000) for _ in range(100000)]

    start_time = time.time()
    unique, dups = process_ids(ids)
    end_time = time.time()

    print(f"Found {len(unique)} unique IDs and {dups} duplicates in {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

## Nested Loop Bottlenecks

In [ ]:
def process_users(user_data):
    # Simulate a large dataset of users
    filtered_users = []


    for user in user_data:
        # Check if user is active and has a valid email
        email = user['email']
        is_active = user['active']

        #  string search for email validation
        if is_active and '@' in email:
            # checking duplicates
            for existing_user in filtered_users:
                if existing_user['email'] == email:
                    break
            else:

                processed_user = {
                    'email': email.lower() + '_processed',
                    'name': user['name'].upper(),
                    'id': str(user['id'])
                }
                filtered_users.append(processed_user)

    return filtered_users

def main():
    # Simulate a large dataset
    users = [
        {'id': i, 'email': f'user{i}@example.com', 'name': f'User {i}', 'active': i % 2 == 0}
        for i in range(100000)
    ]

    # Process the users
    result = process_users(users)
    print(f"Processed {len(result)} users")

if __name__ == "__main__":
    main()

In [ ]:
import re

def process_users(user_data):
    filtered_users = []
    seen_emails = set()  # Track duplicates

    for user in user_data:
        email = user['email']
        if user['active'] and '@' in email and email not in seen_emails:
            processed_user = {
                'email': email.lower() + '_processed',
                'name': user['name'].upper(),
                'id': str(user['id'])
            }
            filtered_users.append(processed_user)
            seen_emails.add(email)  # Mark as seen

    return filtered_users


def main():
    # Simulate a large dataset
    users = [
        {'id': i, 'email': f'user{i}@example.com', 'name': f'User {i}', 'active': i % 2 == 0}
        for i in range(100000)
    ]

    # Process the users
    result = process_users(users)
    print(f"Processed {len(result)} users")

if __name__ == "__main__":
    main()

# SQL Optimization

Let's start with a common mistake that is easy to do in Python SQL. Let's say I want to insert the next 10_000 dates into a `CALENDAR` table that is a single column of dates. Why is this so slow?

In [15]:
import sqlite3

for _ in range(10_000):
    conn = sqlite3.connect("company_operations.db")
    cursor = conn.cursor()
    cursor.execute("""INSERT INTO CALENDAR (CALENDAR_DATE) VALUES
                   ((SELECT DATE(MAX(CALENDAR_DATE),'+1 day') FROM CALENDAR))
                   """)
    conn.commit()
    cursor.close()
    conn.close()



In [ ]:
import pandas as pd
import sqlite3
import time

conn = sqlite3.connect("company_operations.db")

Let's pretend for a moment that this SQL query below took a very long time. Let's see what it recommends.

In [2]:


sql = """
SELECT * FROM WEATHER_MONITOR
WHERE REPORT_DATE = '2021-05-05'
"""

start = time.time()
pd.read_sql(sql, conn)
end = time.time()
print(f"Query took {end - start:.4f} seconds")


Query took 0.0029 seconds


But keep in mind, if the AI recommends an index and other solutions, there are gotchas like write speed going down substantially. These are nuances that you cannot pick up from vibe coding, and is why you should know the subject matter you are prompting.

Here is another example. Let's try to optimize this query.

In [6]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("company_operations.db")

sql = """
SELECT
CUSTOMER_ID,
ORDER_DATE,
QUANTITY,
(SELECT AVG(QUANTITY)
 FROM CUSTOMER_ORDER co3
 WHERE co3.CUSTOMER_ID = co1.CUSTOMER_ID) as avg_customer_quantity
FROM CUSTOMER_ORDER co1
ORDER BY ORDER_DATE
"""

start = time.time()
pd.read_sql(sql, conn)
end = time.time()
print(f"Query took {end - start:.4f} seconds")

,CUSTOMER_ID,ORDER_DATE,QUANTITY,avg_customer_quantity
0,9,2021-01-01,20,110.970149
1,5,2021-01-01,110,104.700855
2,3,2021-01-01,120,103.451327
3,6,2021-01-01,200,100.977444
4,2,2021-01-01,60,103.423423
...,...,...,...,...
1185,9,2021-03-31,70,110.970149
1186,5,2021-03-31,140,104.700855
1187,10,2021-03-31,80,100.080000
1188,9,2021-03-31,20,110.970149


Ideally, we'd like to get a windowing function once the AI recommends a fix. But it may settle for a common table expression or derived table too. This again shows that the AI may propose something that works but may be suboptimal.